In [16]:
import import_ipynb
from library import *

In [17]:
# lấy nhãn và đường dẫn đến ảnh
def get_image_paths(data_dir):
    """
    Lấy tất cả đường link ảnh dựa vào tập (train, test) rồi lưu vào list.

    Args:
        data_dir: Đường dẫn đến thư mục chứa dữ liệu (train, test).

    Returns:
        Một dictionary chứa hai list: train_paths và test_paths, tương ứng với các đường dẫn ảnh trong tập train và test.
    """
    paths = []
    label = []
  # Duyệt qua các thư mục con
    if os.path.isdir(data_dir):
        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)
            label.append(class_name.lower())
            if os.path.isdir(class_dir):
                image_paths = glob.glob(os.path.join(class_dir, '*.jpg')) + glob.glob(os.path.join(class_dir, '*.png'))  # Thêm hỗ trợ cho file png
                paths.extend(image_paths)
    return paths, label

In [18]:
# Hàm phân loại từng ảnh theo nhóm kích thước
def classify_size(size_tuple):
    img = size_tuple
    if img == '224x224':
        return "224x224"
    else:
        return "khac"

In [19]:
class Imagetransform():
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train':transforms.Compose([
            transforms.RandomResizedCrop(resize,scale = (0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

  def __call__(self, img, mode = 'train'):
    return self.data_transform[mode](img)

In [20]:
class Mydataset(data.Dataset):
  def __init__(self, paths, label_map, transform, mode = 'train'):
    self.paths = paths
    self.transform = transform
    self.mode = mode
    self.label_map = label_map
  def __len__(self):
    return len(self.paths)
  def __getitem__(self, idx):
    img_path = self.paths[idx]
    img = Image.open(img_path).convert("RGB")
    img_trans = self.transform(img, mode = self.mode)
    label_name = img_path.split('\\')[-2].lower()
    # chuyển đổi những ảnh có nhãn trùng thành nhãn duy nhất
    if label_name == 'cattle':
      label_name = 'cow'
    elif label_name == 'ladybugs':
      label_name = 'ladybug'
    elif label_name == 'moth':
      label_name = 'moths and butterflies'
    label = self.label_map[label_name]
    return img_trans, label

In [ ]:
class Model():
    def __init__(self, model_pr):
        self.model = model_pr
    def predict(self, image):
        self.model.eval() 
        with torch.no_grad():  
            output = self.model(image) 
            predicted_class = torch.argmax(output, 1)
        return predicted_class.item()  
    def train_model(self, dict_loader, citerior, optimizer, num_epochs = 25):
        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
            for phase in ['train', 'val']:
                if phase == 'train':
                    self.model.train()
                else:
                    self.model.eval()
                epoch_loss = 0.0
                epoch_correct = 0
                if (epoch == 0) and (phase == 'train'):
                    continue
                for inputs, labels in dict_loader[phase]:
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = self.model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = citerior(outputs, labels)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                        epoch_loss += loss.item()*inputs.size(0)
                        epoch_correct += torch.sum(preds == labels.data)
                epoch_loss = epoch_loss / len(dict_loader[phase].dataset)
                epoch_acc = epoch_correct.double() / len(dict_loader[phase].dataset)

                print(f'{phase} loss: {epoch_loss:.4f} acc: {epoch_acc:.4f}')
        return self.model

In [23]:
def load_model(model_path, net):
  load_weights = torch.load(model_path)
  net.load_state_dict(load_weights)
  return net